In [36]:
import csv
import re
import pandas as pd

# Reading in data
in_file = open("data/district_data_23_Billion.csv", 'r')
out_file = open("data/gap_in_spending_per_student.csv", 'w')
writer = csv.writer(out_file)
data = list(csv.reader(in_file))

# Initializing lists/dictionaries
state_revenue_list = []
final = {}
lower = {}
higher = {}

# Getting state and student revenue columns
for i in data[2:]:
    state = i[0]
    revenue = i[13]
    revenue = re.sub("[^0-9]", "", revenue)
    state_revenue_list.append((state,revenue))

# Initializing lower and higher lists
for i in state_revenue_list:
    if i[0] not in lower:
        lower[i[0]] = i[1]
    if i[0] not in higher:
        higher[i[0]] = i[1]
    
# Finding highest and lowest student revenue in each state
for i in state_revenue_list:
    if int(i[1]) > int(higher[i[0]]):
        higher[i[0]] = int(i[1])
    if int(i[1]) < int(lower[i[0]]):
        lower[i[0]] = int(i[1])

# Finding difference
for i in lower:
    final[i] = int(higher.get(i)) - int(lower.get(i))

# Writing to new file
final.pop("District of Columbia")
writer.writerow(["state", "gap_in_spending_per_student"])
for key, value in final.items():
    writer.writerow([key, value])

# Adding state codes to CSV
df = pd.read_csv("data/gap_in_spending_per_student.csv")
state_code = pd.Series(["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"])
df.insert(0,"code",state_code)
df.to_csv("data/gap_in_spending_per_student.csv", index=False)

In [37]:
print(len(final))

50
